In [1]:
import pandas as pd
from threading import Thread
from queue import Queue
import multiprocessing

In [2]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"
csvFileName = "MarketWatchPlus-1398_5_7"
csvFileName = "1382-2-2"

In [3]:
!ls $xcelLocation | grep ".csv" > csvFiles
tmp = !cat csvFiles
csvFiles = [name[:-4] for name in tmp]

In [4]:
def readCsv(xcelLocation, csvFileName):
    df=pd.read_csv(xcelLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

def addDiff(df): 
    diffTable = df.loc[:,["نام","کمترین","بیشترین"]]
    sumdif,count = (0,0)
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.loc[row][1]
        maxim = diffTable.loc[row][2]
        if(minim != 0):
            count += 1
            sumdif += (maxim - minim)/minim
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diffs"] = difs
    df.loc[:,"diffs"] = difs
    return df, diffTable, sumdif, count

In [5]:
df = readCsv(xcelLocation=xcelLocation, csvFileName=csvFileName)
df, diffTable, sumdif, count = addDiff(df)

In [6]:
def readThread(threadname, q, qsum, qcount):
    while not q.empty():
        csvNames = q.get()
        q.task_done()
        
        for csvName in csvNames:
            df = readCsv(xcelLocation=xcelLocation, csvFileName=csvName)
            df, diffTable, sumdif, count = addDiff(df)
            qsum.put(sumdif)
            qcount.put(count)
    print(str(threadname) + " done")

In [ ]:
# print(sumdif/count)
# diffTable
sums, counts = 0,0
batchSize = 10
i = 0
numThread = 8
workers = []

pool = multiprocessing.Pool(processes=numThread)
m = multiprocessing.Manager()
queue = m.Queue()
qsum = m.Queue()
qcount = m.Queue()


while i*batchSize < len(csvFiles):
    if (i+1)*batchSize < len(csvFiles):
        queue.put(csvFiles[i*batchSize:(i+1)*batchSize])
    else:
        queue.put(csvFiles[i*10:])
    i+=1
print(len(csvFiles))
print(queue.qsize())

for i in range(numThread):
#     workers.append(Thread(target=readThread, args=("Thread-" + str(i), queue, qsum, qcount)))
#     workers.append(pool.apply_async(readThread, ("Thread-" + str(i), queue, qsum, qcount,)))
    workers.append(multiprocessing.Process(target=readThread, args=("Thread-" + str(i),
                                                                    queue, qsum, qcount)))
    workers[i].start()

for i in range(numThread):
    workers[i].join()


4227
423
Thread-3 done
Thread-4 done
Thread-1 done
Thread-6 done
Thread-2 done
Thread-7 done
Thread-0 done


In [ ]:
sums = []
counts = []
while not qsum.empty():
    sums.append(qsum.get())
while not qcount.empty():
    counts.append(qcount.get())

In [ ]:
sumss, countss = 0,0
for num in sums:
    sumss += num
for num in counts:
    countss += num
print(sumss/countss)

In [ ]:

# with open('7mivePrice.csv', 'a', encoding="utf-16") as csvfile:
#     excel = csv.writer(csvfile)
#     print("test" + str(name[0]) + " " + str(price[0]))
#     for a, b, c, d in zip(price, name, moredis, weight):
#         excel.writerow([str(a), str(b), now, str(c), str(d)])
         #excel.writerow([str(a).encode(sys.stdout.encoding, errors='replace')] + [str(b).encode(sys.stdout.encoding, errors='replace')])


In [ ]:
xcel57 = readCsv("../", "MarketWatchPlus-1398_5_7")
xcel58 = readCsv("../", "MarketWatchPlus-1398_5_8")
xcel59 = readCsv("../", "MarketWatchPlus-1398_5_9")
_,xcel57,_,_ = addDiff(xcel57)
_,xcel58,_,_ = addDiff(xcel58)
_,xcel59,_,_ = addDiff(xcel59)
col = [col for col in xcel57.columns]
col[0] = "name"
col[1] = "min"
col[2] = "max"
xcel57.columns = col
xcel58.columns = col
xcel59.columns = col

In [ ]:
sumdiffList = []
for index in range(len(xcel57)):
    sumdiff = 0
    name = xcel57.loc[index,"name"]
    row = xcel57.loc[xcel57["name"] == name]
    sumdiff += row.loc[row.index[0], "diffs"]
    row = xcel58.loc[xcel58["name"] == name]
    if len(row) > 0:
        sumdiff += row.loc[row.index[0], "diffs"]
    row = xcel59.loc[xcel59["name"] == name]
    if len(row) > 0:
        sumdiff += row.loc[row.index[0], "diffs"]
    sumdiffList.append((sumdiff, name))

In [ ]:
sumdiffList.sort(reverse=True)

In [ ]:
sumdiffList